In [4]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_VERSION"] = ""
os.environ["OPENAI_API_BASE"] = ""

from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(deployment_name="text-davinci-003", temperature=0)

In [11]:
res1 = llm.predict("1 + 1 = ", stop=("="))
res2 = llm("1 + 1 = ", stop = ["="])
res3 = llm.generate(["1 + 1 = "], stop = ["="]).generations[0][0].text

In [12]:
llm = AzureOpenAI(deployment_name="text-davinci-003", temperature=0, n=2)
llm_result = llm.generate([f"{i}**2 =" for i in range(1, 11)], stop="\n")
print(len(llm_result.generations))
print(len(llm_result.generations[0]))

10
2


In [13]:
llm = AzureOpenAI(deployment_name="text-davinci-003", temperature=0, n=2)
llm_result = llm.generate([f"{i}**2 =" for i in range(1, 11)], stop="\n", n=1)

In [14]:
llm_result.json()

'{"generations": [[{"text": " 1", "generation_info": {"finish_reason": "stop", "logprobs": null}}, {"text": " 4", "generation_info": {"finish_reason": "stop", "logprobs": null}}], [{"text": " 9", "generation_info": {"finish_reason": "stop", "logprobs": null}}, {"text": " 16", "generation_info": {"finish_reason": "stop", "logprobs": null}}], [{"text": " 25", "generation_info": {"finish_reason": "stop", "logprobs": null}}, {"text": " 36", "generation_info": {"finish_reason": "stop", "logprobs": null}}], [{"text": " 49", "generation_info": {"finish_reason": "stop", "logprobs": null}}, {"text": " 64", "generation_info": {"finish_reason": "stop", "logprobs": null}}], [{"text": " 81", "generation_info": {"finish_reason": "stop", "logprobs": null}}, {"text": "", "generation_info": {"finish_reason": "stop", "logprobs": null}}], [], [], [], [], []], "llm_output": {"token_usage": {"prompt_tokens": 40, "total_tokens": 49, "completion_tokens": 9}, "model_name": "text-davinci-003"}, "run": {"run_id

In [3]:
from langchain import PromptTemplate

template = """
我希望你担任顾问，帮忙为公司想名字。
这个公司生产{product}，有什么好名字？
"""

prompt_template = PromptTemplate(
    input_variables=["product"],
    template=template,
)
prompt_template.format(product="运动衬衫")

'\n我希望你担任顾问，帮忙为公司想名字。\n这个公司生产运动衬衫，有什么好名字？\n'

In [3]:
prompt_template = PromptTemplate.from_template(template)
prompt_template.input_variables

['product']

In [ ]:
from langchain.prompts import load_prompt
prompt_template.save("prompt.json")
prompt_template = load_prompt("prompt.json")

## Chain

In [5]:
from langchain.llms import AzureOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = AzureOpenAI(deployment_name="text-davinci-003", temperature=0)
prompt = PromptTemplate(
    input_variables=["input"],
    template="""
    将给定的字符串进行大小写转换。
    例如：
    输入： ABCdef
    输出： abcDEF
    
    输入： AbcDeF
    输出： aBCdEf
    
    输入： {input}
    输出： 
    """,
)

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run("HeLLo"))
print(chain.run({"input": "HeLLo"}))


 hEllO
 hEllO


In [6]:
chain_verbose = LLMChain(llm=llm, prompt=prompt, verbose=True)
print(chain_verbose.run({"input": "HeLLo"}))



> Entering new  chain...
Prompt after formatting:

    将给定的字符串进行大小写转换。
    例如：
    输入： ABCdef
    输出： abcDEF
    
    输入： AbcDeF
    输出： aBCdEf
    
    输入： HeLLo
    输出： 
    

> Finished chain.
 hEllO


In [34]:
from langchain.chains import SimpleSequentialChain

llm = AzureOpenAI(deployment_name="text-davinci-003", temperature=0)
prompt1 = PromptTemplate(
    input_variables=["base"], template="{base}的平方是： "
)
chain1 = LLMChain(llm=llm, prompt=prompt1)

prompt2 = PromptTemplate(input_variables=["input"], template="将{input}写成罗马数字是： ")
chain2 = LLMChain(llm=llm, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
overall_chain.run(3)




> Entering new  chain...


3的平方是9.


IX

> Finished chain.


'\n\nIX'